In [ ]:
import os
import random
import pandas as pd

# Taller 02: Análisis de Sentimiento de Títulos de Noticias Financieras

## Alumnos: *ingresar aquí el nombre de los alumnos a quienes corresponde esta entrega*

## Objetivo

En este taller implementarán un sistema de análisis de sentimiendo que tomando como input el texto escrito en los títulos de noticias financieras, detecta si las mismas son positivas, negativas o neutrales.

El archivo *financial_headlines.csv* contiene los datos con los que deberán trabajar.

La solución debe realizar los siguientes pasos y en el orden en que se presentan:

1. Tokenizar todas las noticias tal cual lo hace la función *tokenize_all* presentada en clases (como ayuda este paso se entrega la función *load_financial_corpus*). Tengan presente que en este corpus a la variable de sentimiento la llamamos "sentiment", mientras que en el utilizado en clase la llamábamos "rank". De este modo, deberán modificar ligeramente la función *tokenize_all* para que funcione en este corpus (este comentario puede valer también para otras funciones vistas en clases que quieran reutilizar a lo largo de este taller).

2. Dividir el corpus en dos conjuntos elegidos al azar. Un conjunto de entrenamiento (que debe contener el 70% de los datos) y un conjunto de testeo (que debe contener el restante 30% de los datos). Para esto se recomienda utilizar *train_test_split* de scikit-learn (vean: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

3. Generar matrices tipo document-term matrix para ambos conjuntos de datos. Para esto pueden (y se recomienda) dejar de lado en todos los conjuntos de datos los tokens menos frecuentes del conjunto de **entrenamiento** (IMPORTANTE: este conteo no debe hacerse sobre el conjunto testeo). Queda a su criterio definir qué es un token poco frecuente. Se pide que **no** realicen la transformación tf-idf.

4. Entrenar un modelo de Random Forest usando como input el conjunto de entrenamiento. Recuerden que la clasificación (esto modelo debe predecir una de las tres clases: "positive", "neutral" y "negative"). Para este paso debe usar los valores por defecto de los hiperparámetros de random forest en scikit learn, salvo para el caso de *n_estimators* que debe ser igual a 400.

5. Evaluar el sistema en el conjunto de testeo tal como lo hicimos en clases, salvo que en este caso, al ser un problema multiclase, se pide que no calculen el área bajo la curva ROC.

Se pide que todo este sistema se encuentre implementado en esta notebook. Además del código presentado, se evaluará que la notebook esté bien explicada, sea prolija, y permita entender de manera simple qué es lo que están haciendo y por qué lo están haciendo.

## Función de ayuda

In [ ]:
def load_financial_corpus():
    fin_data = pd.read_csv("financial_headlines.csv", sep=",", encoding="latin-1")

    for _, row in fin_data.iterrows():
        yield({"id": row["id"], "sentiment": row["sentiment"], "raw_text": row["text"]})

In [ ]:
guia_oleo = load_financial_corpus()

data = []
for e in guia_oleo:
    data.append(e)

## Solución

Aquí deberían ir completando todos los puntos que se piden.

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report

def tokenize(raw_text):
    """ Tokenizes """
    raw_text = raw_text.replace("\'", "'")
    raw_text = raw_text.replace("<br /><br />", "\n")
    raw_text = raw_text.replace("/", " / ")
    sentences = sent_tokenize(raw_text)
    tokens = [e2 for e1 in sentences for e2 in word_tokenize(e1)]  # Nested list comprehension
    tokens = [e.lower() for e in tokens if re.compile("[A-Za-z]").search(e[0])]
    
    return(tokens)


def text_to_bow(tokens):
    return(Counter(tokens))


def tokenize_all(corpus_iterator, size=None, print_every=100):
    """ Processes all review """
    all_reviews = []
    for i, r in tqdm(enumerate(corpus_iterator)):
        tokens = tokenize(r["raw_text"])
        tokens_count = text_to_bow(tokens)
        all_reviews.append({"id": r["id"],
                            "sentiment": r["sentiment"],
                            "tokens": tokens,
                            "tk_count": tokens_count})
        if size and (i+1) == size:
            break
    return(all_reviews)


all_data = tokenize_all(data)


In [ ]:
train_data, test_data = train_test_split([e for e in all_data], test_size=0.3, random_state=42)

In [ ]:
def get_freq_tokens(tokenized_data, min_freq):
    """ Count words frequencies across the corpus"""

    t_freqs = {}

    for i, doc in enumerate(tokenized_data):
        for t in doc["tk_count"]:
            t_freqs[t] = t_freqs.get(t, 0) + doc["tk_count"][t]

    frequent_tokens = set([e for e in t_freqs if t_freqs[e] >= min_freq])

    return(frequent_tokens)


def corpus_to_bow(tokenized_data, freq_tokens):
    """ Shapes data as a pandas DataFrame """

    filt_tokens = []
    for e in tokenized_data:
        filt_tokens.append({k:e["tk_count"][k] for k in e["tk_count"] if k in freq_tokens})

    ids = [e["id"] for e in tokenized_data]
    bag_of_words = pd.DataFrame(filt_tokens, index = ids)
    bag_of_words = bag_of_words.reindex(columns=sorted(freq_tokens)).fillna(0)

    ranks = pd.Series([e["sentiment"] for e in tokenized_data], index = ids)

    return(bag_of_words, ranks)

freq_tokens = get_freq_tokens(train_data, 5)
X_tr, y_tr = corpus_to_bow(train_data, freq_tokens)
X_ts, y_ts = corpus_to_bow(test_data, freq_tokens)


In [ ]:
rf = RandomForestClassifier(n_estimators = 250, n_jobs=-1)

rf.fit(X_tr, y_tr)

y_preds = rf.predict(X_ts)
print(classification_report(y_ts, y_preds))